#Импорт библиотек

In [2]:
!pip install aiogram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.6/570.6 kB 9.4 MB/s eta 0:00:00


In [ ]:
import textwrap

from aiogram import (
    Bot,
    Dispatcher,
    Router,
    types,
)
from aiogram.filters import (
    Command,
    CommandStart,
)
from aiogram.types import BotCommand
from aiogram.types.callback_query import CallbackQuery
from aiogram.utils.keyboard import InlineKeyboardBuilder

# Telegram bot

Бот загружает заранеее подготовленные модели с диска

Команды бота:

`/start `- Вывести приветственное сообщение с описанием команд бота

`/model` - Вывести список доступных моделей в виде кнопок. Доступные модели ('unigram', 'unigram_bigram', 'bigram', 'trigram')

`/generate` - Отдать тэг для отправленной в бот новостной статьи

`/checkmodel` - Вывести загруженную в данный момент модель

`/help` - Вывести список доступных команд

In [ ]:
class ClassificationBot:

    def __init__(self, token, models, vectorizers, encoder, supported_models):

        self.token = token
        self.bot = Bot(token)

        self.dispatcher = Dispatcher() # Создаем диспетчер для обработки сообщений

        # сохраняем модели, векторизаторы, кодировщик и поддерживаемые модели
        self.models = models
        self.vectorizers = vectorizers
        self.encoder = encoder
        self.supported_models = supported_models
        self.active_model = supported_models[0] # По умолчанию активной моделью становится первая в списке

    async def initialize_router(self):
        # маршрутизатор
        router = Router(name="classification")

        @router.message(CommandStart()) # Обработчик команды /start
        async def start_handler(message: types.Message) -> None:

            welcome_message = """
            Добро пожаловать!
            Бот умеет подбирать топик для новостных статей.
            ...
            """

            if message.from_user:
                await message.answer(
                    textwrap.dedent(welcome_message),
                )

        @router.message(Command("model")) # Обработчик команды /model
        async def select_model(message: types.Message) -> None:

            builder = InlineKeyboardBuilder()

            for model in self.supported_models:
                builder.row(
                    types.InlineKeyboardButton(
                        text=model,
                        callback_data=model,
                    )
                )

            if message.from_user:
                await message.answer(
                    "Выберите модель:",
                    reply_markup=builder.as_markup(
                        resize_keyboard=True,
                    ),
                )

        @router.callback_query() # Обработчик нажатий на клавиатуру для изменения модели
        async def change_active_model(callback: CallbackQuery, bot: Bot) -> None:

            self.active_model = callback.data

            await callback.answer(
                f"Выбрана модель: {self.active_model}",
            )
            await bot.delete_message(
                chat_id=callback.message.chat.id,
                message_id=callback.message.message_id,
            )

        @router.message(Command("checkmodel")) # Обработчик команды /checkmodel
        async def check_model(message: types.Message) -> None:

            if message.from_user:
                await message.answer(
                    f"Активная модель: {self.active_model}",
                )

        @router.message(Command("generate")) # Обработчик команды /generate
        @router.message() # Обработчик всех сообщений бота
        async def check_model(message: types.Message) -> None:

            user_input = message.text.lstrip("/generate") # Удаляем лишние пробелы

            user_input = preprocess_text(user_input) # Обрабатываем текст

            classifier = self.models[self.active_model] # Получаем модель
            vectorizer = self.vectorizers[self.active_model] # Получаем векторизатор

            test_features = vectorizer.transform([user_input]) # Преобразуем текст в вектор
            predicted_label = classifier.predict(test_features) # Предсказываем класс

            label = self.encoder.classes_[predicted_label] # Декодируем предсказанный класс

            if message.from_user:
                await message.answer(
                    f"Предсказанный топик: {label}",
                )

        self.dispatcher.include_router(router=router) # Добавляем маршрутизатор в диспетчер

    async def run(self): # Запуск бота
        commands_dict: dict[str, str] = {
            "/start": "Начать работу с ботом",
            "/model": "Выбрать модель",
            "/checkmodel": "Вывести активную на даный момент модель",
            "/generate": "Сгенерировать топик для новостной статьи",
        }

        main_menu_commands = [
            BotCommand(command=command, description=description)
            for command, description in commands_dict.items()
        ]

        await self.bot.set_my_commands(main_menu_commands) # Устанавливаем команды для бота
        await self.dispatcher.start_polling(self.bot) # Запускаем бота

#Токен

In [ ]:
# вставить токен для тг бота
TG_TOKEN = ""

#Загрузка моделей

In [ ]:
# загрузим модели с диска
test_classifiers, test_vectorizers, test_encoder = load_models()
supported_models = ['unigram', 'unigram_bigram', 'bigram', 'trigram']

#Классификация

In [ ]:
# инициализация бота
bot = ClassificationBot(
    TG_TOKEN,
    test_classifiers,
    test_vectorizers,
    test_encoder,
    supported_models,
)

await bot.initialize_router()
await bot.run()